In [ ]:
# 1. Import các thư viện cần thiết
import numpy as np
import pickle
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from cso import CSO

# 2. Load dữ liệu PhoBERT embeddings và nhãn
X_train = np.load('models/phobert_svm_model/X_train_emb.npy')
y_train = np.load('models/phobert_svm_model/y_train.npy')
X_test = np.load('models/phobert_svm_model/X_test_emb.npy')
y_test = np.load('models/phobert_svm_model/y_test.npy')

# 3. Định nghĩa các loại kernel cho SVM
kernel_map = ['linear', 'rbf', 'poly']

# Các giá trị rời rạc cho C
C_values = [0.1, 1, 10]

# 4. Định nghĩa hàm fitness cho SVM
def svm_fitness(X):
    # X: [C_idx, kernel_idx]
    C_idx = int(np.clip(round(X[0]), 0, len(C_values)-1))
    kernel_idx = int(np.clip(round(X[1]), 0, len(kernel_map)-1))
    C = C_values[C_idx]
    kernel = kernel_map[kernel_idx]
    try:
        clf = SVC(C=C, kernel=kernel)
        clf.fit(X_train, y_train)
        y_pred = clf.predict(X_test)
        acc = accuracy_score(y_test, y_pred)
        return -acc
    except Exception:
        return 1.0

# Giới hạn cho các tham số [C_idx, kernel_idx]
bounds = [(0, len(C_values)-1), (0, len(kernel_map)-1)]

# 6. Chạy tối ưu hóa bằng Cuckoo Search
cso = CSO(fitness=svm_fitness, P=10, n=2, bound=bounds, Tmax=5, verbose=True, plot=True)
cso.execute()
best_params = cso.best
print('Best hyperparameters:', best_params)
print('Best C:', C_values[int(round(best_params[0]))])
print('Best kernel:', kernel_map[int(round(best_params[1]))])

# 7. Đánh giá lại mô hình SVM với tham số tối ưu
C_opt = C_values[int(round(best_params[0]))]
kernel_opt = kernel_map[int(np.clip(round(best_params[1]), 0, len(kernel_map)-1))]
clf_opt = SVC(C=C_opt, kernel=kernel_opt)
clf_opt.fit(X_train, y_train)
y_pred_opt = clf_opt.predict(X_test)
acc_opt = accuracy_score(y_test, y_pred_opt)
print('Optimized SVM accuracy:', acc_opt)

# Lưu mô hình đã huấn luyện
with open('models/phobert_svm_model/svm_cso_optimized.pkl', 'wb') as f:
    pickle.dump(clf_opt, f)

# Lưu các tham số tối ưu và độ chính xác
results = {
    'best_params': best_params,
    'best_C': C_opt,
    'best_kernel': kernel_opt,
    'accuracy': acc_opt
}
with open('models/phobert_svm_model/svm_cso_results.pkl', 'wb') as f:
    pickle.dump(results, f)
